In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [2]:
from divtel import *
#from divtel.ArrayConfig import *
from pathlib import Path

In [12]:
prod='PROD6'
outdir='./Prod6/simtel'
site='Paranal'     #allowed values: LaPalma, Paranal

In [4]:
alt=70  #[30,50,70]
az=180    #[0,180]
if az==0:
    pointing='North'
elif az==180:
    pointing='South'

In [5]:
if site == 'LaPalma':
    camera = 'Nectar'
elif site == 'Paranal':
    camera = 'Flash'

In [8]:
array = LoadConfig(f"CTA-{prod}_TelescopePositions{site}.txt")

In [9]:
divergence=[0.0022,0.0043,0.008,0.01135,0.01453]

In [10]:
configs=['CTA-PROD6-LST',f'CTA-PROD6-MST-{camera}Cam','CTA-PROD6-SST']
LST_config=configs[0]
MST_config=configs[1]
SST_config=configs[2]

In [14]:
#path = Path(f"{outdir}")
#path.mkdir(exist_ok=True)
#path = Path(f"{outdir}/{site}")
#path.mkdir(exist_ok=True)
#path = Path(f"{outdir}/{site}/{pointing}_pointing")
#path.mkdir(exist_ok=True)

In [15]:
if site=='LaPalma':
    geosite='La_Palma'
    atm='NORTH'
    hemisphere='North'
    array_config_variant=f"LST/{camera}Cam at CTA {hemisphere} with Alpha+ layout"
elif site =='Paranal':
    geosite=site
    atm='SOUTH'
    hemisphere='South'
    array_config_variant=f"LST/{camera}Cam/SST at CTA {hemisphere} with Alpha+ layout"

In [17]:
for div in divergence: 
    #print(div)
    array.divergent_pointing(div = div, az = az, alt = alt)
    array.table.units = "deg"
    table = array.table
    filename = f"CTA-{prod}-{site}-div{div}_Zd{90 - alt}_{pointing}Pointing.cfg"
    with open(filename, 'w') as f:
        f.write("#ifndef NO_GSL_RNG\n")
        f.write("     random_generator = mt19937 % Faster than ranlux. Will fail if not compiled with -DWITH_GSL_RNG.\n")
        f.write("#endif\n")
        f.write("\n")
        f.write(f"array_config_name = {site}-Alpha-{prod}\n")
        f.write(f"array_config_variant = {array_config_variant}\n")
        f.write("array_config_version = 2022-07-04\n")
        f.write("\n")
        f.write("% What transmission option to use (see CTA-PROD6-site.cfg):\n")
        f.write(f"#define CTA_{geosite.upper()} 1\n")
        f.write(f"#define ATMOSPHERE_{atm} 1\n")
        #f.write(" #define NAVY_MARITIME 1\n")
        f.write("#define LOW_EXTINCTION 1\n")
        f.write("\n")
        f.write("#ifndef TELESCOPE\n")
        f.write("#   define TELESCOPE 0\n")
        f.write("#endif\n")
        f.write("\n")
        f.write('#if TELESCOPE == 0\n')
        f.write('   TELESCOPE_THETA={:.2f}\n'.format(90 - array.pointing["alt"].value))
        f.write('   TELESCOPE_PHI={:.2f}\n'.format(array.pointing["az"].value))
        f.write('\n% Global and default configuration for things missing in telescope-specific config.\n')
        f.write(f'#  include <CTA-PROD6-dummy-tel.cfg>\n')
        for n, tel in enumerate(table, 1):
            #zd = 90 - tel["alt"]
            f.write('\n#elif TELESCOPE == {:d}\n'.format(n))
            if tel["focal"]==28:
                if tel['id'] ==1 and 'LaPalma' in filename:
                    f.write(f"#  include <CTA-PROD6-LST-prototype.cfg>\n")
                else:
                    f.write(f"#  include <{LST_config}.cfg>\n")
            elif tel["focal"]==16:
                f.write(f"#  include <{MST_config}.cfg>\n")
            elif tel["focal"]==5.6:
                f.write(f'#  include <{SST_config}.cfg>\n')
                
            f.write("   TELESCOPE_THETA={:.2f}\n".format(90 - tel["alt"]))
            f.write("   TELESCOPE_PHI={:.2f}\n".format(360 - tel["az"]))
            f.write("\n")
        f.write("#else\n")
        f.write("   Error Invalid telescope for CTA-PROD6 Paranal baseline/extended configuration.\n")
        f.write("#endif\n")
        f.write("trigger_telescopes = 2 % We apply loose stereo trigger immediately\n")
        f.write("% To be more strict we need a matching array trigger definition.\n")
        f.close()